In [36]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.llms.ollama import Ollama

In [37]:
template = """Question: {question}


Answer: Let's think step by step."""


prompt = ChatPromptTemplate.from_template(template)


model = Ollama(model="llama3")


chain = prompt | model

In [47]:
our_prompt = """
i have this following class with two functions
import amadeus

def get_flight_offers(self, origin, destination, departure_date, n_adults):
    response = self.amadeus.shopping.flight_offers_search.get(
        originLocationCode=origin,
        destinationLocationCode=destination,
        departureDate=departure_date,
        adults=n_adults
    )
    return response.data

def get_hotel_by_city(self, city_code):
    response = self.amadeus.reference_data.locations.hotels.by_city.get(cityCode=city_code)
    return response.data


(dates in YYYY-MM-DD format)
based on the function definitions, can you pick the function definition that matches the user query that I am going to give, give the function call for it according to the information in the query? (do not give  ANY explanations, just the function call). You **must** give the response in JSON format where there are two different objects, one for hotels and the other for flights.
For example:
{
"task": "get_flight_offers",
"params": {
"origin": "JFK",
"destination": "LAX",
"departure_date": "2024-11-11",
"n_adults": 1
}
}

"""

user_query = "Give flight details from ATH to LAX on the 11th of november, 2024"

In [39]:
response = chain.invoke({"question": our_prompt+user_query})

In [40]:
print(response)

{
"task": "get_hotel_by_city",
"params": {
"city_code": "PAR"
}
}


In [41]:
import json 

data = json.loads(response)
data

{'task': 'get_hotel_by_city', 'params': {'city_code': 'PAR'}}

In [44]:
import amadeus_client

result = ""

if data['task']=='get_flight_offers':
    origin = data['params']['origin']
    destination = data['params']['destination']
    departure_date = data['params']['departure_date']
    n_adults = data['params']['n_adults']
    flight_data = amadeus_client.getting_flight_offers(origin = origin, destination = destination, departure_date = departure_date, n_adults = n_adults)
    result = flight_data

# print(flight_data)
else:
    city_code = data['params']['city_code']
    hotel_data =amadeus_client.getting_hotels(city_code = city_code)
    result = hotel_data
    

ModuleNotFoundError: No module named 'amedeus_client'

In [49]:
result = """
[{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2024-11-01', 'lastTicketingDateTime': '2024-11-01', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT18H30M', 'segments': [{'departure': {'iataCode': 'ATH', 'at': '2024-11-01T02:35:00'}, 'arrival': {'iataCode': 'IST', 'at': '2024-11-01T05:05:00'}, 'carrierCode': 'TK', 'number': '1842', 'aircraft': {'code': '32B'}, 'operating': {'carrierCode': 'TK'}, 'duration': 'PT1H30M', 'id': '45', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'IST', 'at': '2024-11-01T08:10:00'}, 'arrival': {'iataCode': 'LAX', 'terminal': 'B', 'at': '2024-11-01T12:05:00'}, 'carrierCode': 'TK', 'number': '179', 'aircraft': {'code': '789'}, 'operating': {'carrierCode': 'TK'}, 'duration': 'PT13H55M', 'id': '46', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1346.78', 'base': '732.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amount': '0.00', 'type': 'TICKETING'}], 'grandTotal': '1346.78'}, 'pricingOptions': {'fareType': ['PUBLISHED'], 'includedCheckedBagsOnly': True}, 'validatingAirlineCodes': 
['TK'], 'travelerPricings': [{'travelerId': '1', 'fareOption': 'STANDARD', 'travelerType': 'ADULT', 'price': {'currency': 'EUR', 'total': '673.39', 'base': '366.00'}, 'fareDetailsBySegment': [{'segmentId': '45', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': 
{'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}, {'segmentId': '46', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 
'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}]}, {'travelerId': '2', 'fareOption': 'STANDARD', 'travelerType': 'ADULT', 'price': {'currency': 'EUR', 'total': '673.39', 'base': '366.00'}, 'fareDetailsBySegment': [{'segmentId': '45', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 
'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}, {'segmentId': '46', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}]}]}, {'type': 'flight-offer', 'id': '2', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2024-11-01', 'lastTicketingDateTime': '2024-11-01', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT24H10M', 'segments': 
[{'departure': {'iataCode': 'ATH', 'at': '2024-11-01T02:35:00'}, 'arrival': {'iataCode': 'IST', 'at': '2024-11-01T05:05:00'}, 'carrierCode': 'TK', 'number': '1842', 'aircraft': {'code': '32B'}, 'operating': {'carrierCode': 'TK'}, 'duration': 'PT1H30M', 'id': '13', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 
'IST', 'at': '2024-11-01T13:50:00'}, 'arrival': {'iataCode': 'LAX', 'terminal': 'B', 'at': '2024-11-01T17:45:00'}, 'carrierCode': 'TK', 'number': '9', 'aircraft': {'code': '77W'}, 'operating': {'carrierCode': 'TK'}, 'duration': 'PT13H55M', 'id': '14', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1346.78', 'base': '732.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amount': '0.00', 'type': 'TICKETING'}], 'grandTotal': '1346.78'}, 'pricingOptions': {'fareType': ['PUBLISHED'], 'includedCheckedBagsOnly': True}, 'validatingAirlineCodes': ['TK'], 'travelerPricings': [{'travelerId': '1', 'fareOption': 'STANDARD', 'travelerType': 'ADULT', 'price': {'currency': 'EUR', 'total': '673.39', 'base': '366.00'}, 'fareDetailsBySegment': [{'segmentId': '13', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 
'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 
'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}, {'segmentId': '14', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}]}, {'travelerId': '2', 'fareOption': 'STANDARD', 'travelerType': 'ADULT', 'price': {'currency': 'EUR', 'total': '673.39', 'base': '366.00'}, 'fareDetailsBySegment': [{'segmentId': '13', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}, {'segmentId': '14', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': 
{'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}]}]}, {'type': 'flight-offer', 'id': '3', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2024-11-01', 'lastTicketingDateTime': '2024-11-01', 'numberOfBookableSeats': 8, 'itineraries': [{'duration': 'PT24H', 'segments': [{'departure': {'iataCode': 'ATH', 'at': '2024-11-01T21:05:00'}, 'arrival': {'iataCode': 
'IST', 'at': '2024-11-01T23:40:00'}, 'carrierCode': 'TK', 'number': '1846', 'aircraft': {'code': '32B'}, 'operating': {'carrierCode': 'TK'}, 'duration': 'PT1H35M', 'id': '7', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'IST', 'at': '2024-11-02T08:10:00'}, 'arrival': {'iataCode': 'LAX', 'terminal': 'B', 'at': '2024-11-02T12:05:00'}, 'carrierCode': 'TK', 'number': '179', 'aircraft': {'code': '789'}, 'operating': {'carrierCode': 'TK'}, 'duration': 'PT13H55M', 'id': '8', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1346.78', 'base': '732.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amount': '0.00', 'type': 'TICKETING'}], 'grandTotal': '1346.78'}, 'pricingOptions': {'fareType': ['PUBLISHED'], 'includedCheckedBagsOnly': True}, 'validatingAirlineCodes': ['TK'], 'travelerPricings': [{'travelerId': '1', 'fareOption': 'STANDARD', 'travelerType': 'ADULT', 'price': {'currency': 'EUR', 'total': '673.39', 'base': '366.00'}, 'fareDetailsBySegment': [{'segmentId': '7', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedCheckedBags': {'quantity': 2}, 'amenities': [{'description': 'BAG INCLUDED', 'isChargeable': False, 'amenityType': 'BAGGAGE', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'MEAL SERVICE', 'isChargeable': False, 'amenityType': 'MEAL', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'STANDART SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'REFUNDABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'CHANGEABLE TICKET', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'ONLINE MESSAGE RIGHT', 'isChargeable': False, 'amenityType': 'ENTERTAINMENT', 'amenityProvider': {'name': 'BrandedFare'}}, {'description': 'FRONT SEAT SELECTION', 'isChargeable': True, 'amenityType': 'BRANDED_FARES', 'amenityProvider': {'name': 'BrandedFare'}}]}, {'segmentId': '8', 'cabin': 'ECONOMY', 'fareBasis': 'QLN2PXOW', 'brandedFare': 'RS', 'brandedFareLabel': 'RESTRICTED', 'class': 'Q', 'includedChec
"""
our_prompt2 = f"""I have provided to you with user question and the JSON answer. Can you present it in a normal english textual format without using JSON format. with only the important information
: User question: {user_query}. Result: {result}. Can you give the answer in human understandable format?"""

response2 = chain.invoke({"question": our_prompt2})

In [50]:
response2

"A JSON response!\n\nLet me help you break it down into a human-readable format.\n\n**Flight Information**\n\n* Flight ID: 3\n* Source: GDS (Global Distribution System)\n* Instant ticketing required: No\n* Non-homogeneous: No\n* One-way: No\n* Last ticketing date: November 1, 2024\n\n**Itinerary**\n\n* Duration: 24 hours\n* Segments:\n\t+ Departure: Athens (ATH) on November 1, 2024 at 21:05\n\t+ Arrival: Istanbul (IST) on November 1, 2024 at 23:40\n\t+ Flight number: TK1846\n\t+ Aircraft code: 32B\n* Segment 2:\n\t+ Departure: Istanbul (IST) on November 2, 2024 at 08:10\n\t+ Arrival: Los Angeles (LAX) on November 2, 2024 at 12:05\n\t+ Flight number: TK179\n\t+ Aircraft code: 789\n\n**Price**\n\n* Currency: EUR\n* Total price: €1,346.78\n* Base price: €732.00\n* Fees:\n\t+ Supplier fee: €0.00\n\t+ Ticketing fee: €0.00\n\n**Pricing Options**\n\n* Fare type: PUBLISHED\n* Included checked bags only: Yes\n\n**Validating Airline Codes**\n\n* TK (Turkish Airlines)\n\n**Traveler Pricing**\n\n*

In [51]:
print(response2)

A JSON response!

Let me help you break it down into a human-readable format.

**Flight Information**

* Flight ID: 3
* Source: GDS (Global Distribution System)
* Instant ticketing required: No
* Non-homogeneous: No
* One-way: No
* Last ticketing date: November 1, 2024

**Itinerary**

* Duration: 24 hours
* Segments:
	+ Departure: Athens (ATH) on November 1, 2024 at 21:05
	+ Arrival: Istanbul (IST) on November 1, 2024 at 23:40
	+ Flight number: TK1846
	+ Aircraft code: 32B
* Segment 2:
	+ Departure: Istanbul (IST) on November 2, 2024 at 08:10
	+ Arrival: Los Angeles (LAX) on November 2, 2024 at 12:05
	+ Flight number: TK179
	+ Aircraft code: 789

**Price**

* Currency: EUR
* Total price: €1,346.78
* Base price: €732.00
* Fees:
	+ Supplier fee: €0.00
	+ Ticketing fee: €0.00

**Pricing Options**

* Fare type: PUBLISHED
* Included checked bags only: Yes

**Validating Airline Codes**

* TK (Turkish Airlines)

**Traveler Pricing**

* Traveler ID: 1
* Fare option: STANDARD
* Traveler type: A